In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import json
import pandas as pd
from pathlib import Path
from copy import deepcopy
from functools import partial

from bellek.qa.llm import make_question_answer_func
from bellek.qa.ablation import answer_question_standard
from bellek.utils import set_seed, jprint
from bellek.musique.multihop import benchmark

set_seed(89)

In [3]:
from tqdm.auto import tqdm
tqdm.pandas()

# Dataset

In [4]:
from bellek.musique.constants import ABLATION_RECORD_IDS

df = pd.read_json('../../data/generated/musique-evaluation/dataset.jsonl', orient='records', lines=True)

# Take ablation subset
df = df.set_index('id', drop=False).loc[ABLATION_RECORD_IDS].copy().reset_index(drop=True)

# Load question decomposition
qd_df = pd.read_json('../../data/generated/musique-evaluation/question-decomposition.jsonl', orient='records', lines=True)
df = pd.merge(df.drop(columns=['question', 'question_decomposition']), qd_df, on='id', suffixes=('', ''))

print(len(df))
df.head()

100


,id,paragraphs,question,question_decomposition,answer,answer_aliases,answerable,answers
0,2hop__575188_342798,"[{'idx': 0, 'title': 'Liliana Mumy', 'paragrap...",Who is the child of Mahmoud Mirza's father?,"[{'id': 575188, 'question': 'Mahmoud Mirza >> ...",Ahmad Shah Qajar,[Ahmad Shah Qajar],True,[Ahmad Shah Qajar]
1,2hop__731584_700117,"[{'idx': 0, 'title': 'KAPE', 'paragraph_text':...",In which county is the city to which KKVU is l...,"[{'id': 731584, 'question': 'KKVU >> licensed ...",Berrien County,[Berrien County],True,[Berrien County]
2,2hop__690412_526810,"[{'idx': 0, 'title': 'Cabramatta Creek', 'para...",For what river does the river on which Pa Sak ...,"[{'id': 690412, 'question': 'Pa Sak Jolasid Da...",Chao Phraya River,[Chao Phraya River],True,[Chao Phraya River]
3,2hop__263638_69048,"[{'idx': 0, 'title': 'Michael J. Barron', 'par...",Who is the Chief Judge of the Tebesa Nemine's ...,"[{'id': 263638, 'question': 'Tebesa Nemine >> ...",Honorable Justice Abiodun Smith,[Honorable Justice Abiodun Smith],True,[Honorable Justice Abiodun Smith]
4,2hop__142842_68489,"[{'idx': 0, 'title': 'Perfect Night: Live in L...",Who did the performer of Night Rocker play on ...,"[{'id': 142842, 'question': 'Which performer r...",Snapper Foster,[Snapper Foster],True,[Snapper Foster]


In [6]:
df['paragraphs'] = df['paragraphs'].map(lambda ps: [p for p in ps if p['is_supporting']])

# Definitions

In [7]:
perfect_retrieval_func = lambda docs, query: [doc for doc in docs if doc['is_supporting']]

In [8]:
qa_func = answer_question_standard

In [22]:
from bellek.jerx.fewshot.llm import DEFAULT_JERX_SYSTEM_MESSAGE_FOR_LLAMA, DEFAULT_FEW_SHOT_EXAMPLE_MESSAGES
from bellek.jerx.fewshot.llm import make_kg_triplet_extract_fn

PREFIX_MESSAGES = [
    dict(role="system", content=DEFAULT_JERX_SYSTEM_MESSAGE_FOR_LLAMA),
    *DEFAULT_FEW_SHOT_EXAMPLE_MESSAGES,
]

In [19]:
default_completion_params = {
    "temperature": 0.1
}

def make_paragraph_replacer(model: str, prefix_messages: list[dict], completion_params: dict = default_completion_params):
    extract_kg_triplets = make_kg_triplet_extract_fn(model=model, prefix_messages=prefix_messages, completion_params=completion_params)

    def replace_paragraphs(row):
        new_paragraphs = []
        for p in row['paragraphs']:
            p = deepcopy(p) 
            triplets_str = '\n'.join(" | ".join(triplet) for triplet in extract_kg_triplets(p['paragraph_text']))
            p['paragraph_text'] = '\n'.join(["# Entity-relation-entity triplets", triplets_str])
            new_paragraphs.append(p)
        row['paragraphs'] = new_paragraphs
        return row
    
    return replace_paragraphs

# Experiments

In [ ]:
N_RUNS = 3

results = []

## llama-zero-shot

In [15]:
rp_zs = make_paragraph_replacer('llama-3-8b-tgi', PREFIX_MESSAGES[:1])

for i in range(1, N_RUNS+1):
    df_llama_zs = df.progress_apply(rp_zs, axis=1) 
    df_llama_zs, scores = benchmark(df_llama_zs, qa_func, perfect_retrieval_func, ignore_errors=True)
    results.append({**scores, "retrieval": "groundtruth", "context": "triplets", "jerx": "llama-zero-shot", "run": i})
    jprint(scores)

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

{
  "exact_match": 0.52,
  "f1": 0.57,
  "fuzzy_match": 0.58
}


  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

{
  "exact_match": 0.54,
  "f1": 0.58,
  "fuzzy_match": 0.58
}


  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

{
  "exact_match": 0.5,
  "f1": 0.5413333333333333,
  "fuzzy_match": 0.53
}


## llama-few-shot

In [23]:
rp_fs = make_paragraph_replacer('llama-3-8b-tgi', PREFIX_MESSAGES)

for i in range(1, N_RUNS+1):
    df_llama_fs = df.progress_apply(rp_fs, axis=1) 
    df_llama_fs, scores = benchmark(df_llama_fs, qa_func, perfect_retrieval_func, ignore_errors=True)
    results.append({**scores, "retrieval": "groundtruth", "context": "triplets", "jerx": "llama-few-shot", "run": i})
    jprint(scores)

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

{
  "exact_match": 0.49,
  "f1": 0.5732698412698413,
  "fuzzy_match": 0.59
}


  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

{
  "exact_match": 0.5,
  "f1": 0.5849304029304029,
  "fuzzy_match": 0.58
}


  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

{
  "exact_match": 0.53,
  "f1": 0.6037096237096238,
  "fuzzy_match": 0.61
}


## llama-sft

In [31]:
rp_sft_zs = make_paragraph_replacer('llama-3-8b-sft-tgi', PREFIX_MESSAGES[:1])

for i in range(1, N_RUNS+1):
    df_llama_sft_zs = df.progress_apply(rp_sft_zs, axis=1) 
    df_llama_sft_zs, scores = benchmark(df_llama_sft_zs, qa_func, perfect_retrieval_func, ignore_errors=True)
    results.append({**scores, "retrieval": "groundtruth", "context": "triplets", "jerx": "llama-sft-zs", "run": i})
    jprint(scores)

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

{
  "exact_match": 0.49,
  "f1": 0.5356666666666667,
  "fuzzy_match": 0.55
}


  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

{
  "exact_match": 0.43,
  "f1": 0.49499999999999994,
  "fuzzy_match": 0.49
}


  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

{
  "exact_match": 0.45,
  "f1": 0.5006666666666667,
  "fuzzy_match": 0.52
}


In [ ]:
rp_sft_fs = make_paragraph_replacer('llama-3-8b-sft-tgi', PREFIX_MESSAGES)

for i in range(1, N_RUNS+1):
    df_llama_sft_fs = df.progress_apply(rp_sft_fs, axis=1) 
    df_llama_sft_fs, scores = benchmark(df_llama_sft_fs, qa_func, perfect_retrieval_func, ignore_errors=True)
    results.append({**scores, "retrieval": "groundtruth", "context": "triplets", "jerx": "llama-sft-fs", "run": i})
    jprint(scores)

# Report

In [27]:
pd.options.display.float_format = '{:,.3f}'.format

In [28]:
report_df = pd.DataFrame.from_records(results, columns=['run', 'context', 'retrieval', 'jerx', 'exact_match', 'fuzzy_match', 'f1'])
report_df.drop(columns=['context', 'retrieval', 'fuzzy_match'])

,run,jerx,exact_match,f1
0,1,llama-zero-shot,0.520,0.570
1,2,llama-zero-shot,0.540,0.580
2,3,llama-zero-shot,0.500,0.541
3,1,llama-few-shot,0.490,0.573
4,2,llama-few-shot,0.500,0.585
5,3,llama-few-shot,0.530,0.604


In [30]:
report_df[['jerx', 'exact_match', 'f1']].groupby('jerx').mean()

,exact_match,f1
jerx,,
llama-few-shot,0.507,0.587
llama-zero-shot,0.520,0.564
